## imports

In [2]:
import os
import contextlib
import enum
import json
import os
import pathlib
import typing as tp
import uuid
from tqdm import tqdm
import copy
from collections import Counter
from  itertools import chain
import ast
import pandas as pd
import random


tqdm.pandas()

#nlp
from nltk import WordNetLemmatizer
from nltk.corpus import stopwords
from razdel import sentenize
import nltk
import re
import string

nltk.download('wordnet')
nltk.download('omw-1.4')
#nltk.download('stopwords')

[nltk_data] Downloading package wordnet to /home/alrabosh/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /home/alrabosh/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


True

## preprocessing

### courseworks se

In [3]:
df = pd.read_csv("data/se_spbu/works.csv")

In [4]:
def string_list_to_list(string):
    return string[2:-1].split("', '")

In [5]:
if df.texts[0][0] == "[":
    df['texts'] = df.texts.apply(string_list_to_list)
    df['texts'] = df["texts"].apply(lambda x: x[1:])

In [6]:
from ast import literal_eval
df['merged_text']=df.texts.progress_apply(lambda x: "\n".join(x))

100%|██████████| 419/419 [00:00<00:00, 40518.60it/s]


In [7]:
def line_break_resolution(text):
    return text.replace("-\n", "")

def delete_links(text):  # useless???
    return re.sub(r'\[.*?\]', "", text)

def delete_titles(text):
    return re.sub(r'\n[0-9].*\n', "\n", text) 

def double_tire(text):
    return text.replace("--", "-")

def delete_uf(text):
    return text.replace(u"\\uf113", " ")

def delete_special_chars(text):
    chars = '●•-'
    return text.translate(str.maketrans('', '', chars))

def delete_multiple_ns(text):
    while "\n\n" in text:
        text = text.replace("\n\n", "\n")
    return text

def delete_ns(text):
    return text.replace("\n", " ")

def images_description(text):
    text = text.replace("Рис.", "Рисунок")
    text = text.replace("рис.", "рисунок")
    return text



In [8]:
def clean_text(text):
    text = delete_multiple_ns(text)
    text = line_break_resolution(text)
    text = double_tire(text)
    text = delete_titles(text)
    text = delete_special_chars(text)
    text = images_description(text)
    text = delete_ns(text)
    text = delete_uf(text)
    return text

def split_text(text):
    splitted = text.split(".")
    splitted = [sen.strip() + "." for sen in splitted]
    return list(sentenize(text))

def split_and_clean(text):
    return split_text(clean_text(text))

In [9]:
clean_texts = df.merged_text.progress_apply(split_and_clean)

100%|██████████| 419/419 [00:02<00:00, 180.37it/s]


In [10]:
sentences = []
for text in clean_texts:
    for sen in text:
        sentences.append(sen.text)
print(len(sentences))

44695


In [11]:
def filter_short_sentences(sents):
    return [sen for sen in sents if len(sen) > 15]

def filter_long_sentences(sents):
    return [sen for sen in sents if len(sen) < 1000]

def filter_tables(sents):
    return list(filter(lambda x: "таблица" not in x and "Таблица" not in x, sents))

def filter_double_dots(sents):
    return [sen for sen in sents if ". ." not in sen]

def filter_painting(sents):
    return [sen for sen in sents if "траница" not in sen]
    
def filter_pages(sents):
    return [sen for sen in sents if "исунок" not in sen]

def balanced_brackets(s):
    pairs = {"{": "}", "(": ")", "[": "]"}
    stack = []
    for c in s:
        if c not in "{[(}])":
            continue
        if c in "{[(":
            stack.append(c)
        elif stack and c == pairs[stack[-1]]:
            stack.pop()
        else:
            return False
    return len(stack) == 0

def is_not_strange(text):
    special_characters = "@#^&*+_=<✓α>/≡≡Σ∈≤"
    return balanced_brackets(text) and not any(c in special_characters for c in text)

def make_text(sents):
    sents = filter_short_sentences(sents)
    sents = filter_long_sentences(sents)
    sents = filter_tables(sents)
    sents = filter_double_dots(sents)
    
    triples = [sents[i:i+1] for i in range(0, len(sents) - 1)]
    texts = [" ".join(triple) for triple in triples]
    return [text for text in texts if is_not_strange(text)]

In [12]:
texts = make_text(sentences)

In [13]:
sentences = list(clean_texts.explode().apply(lambda x: x.text))
sentences = [sen for sen in sentences if sen]

In [14]:
sentences = texts

In [15]:
len(sentences)

39618

In [253]:
random.sample(sentences,1000)

['Работа с Dockerконтейнерами поддержана во многих популярных системах непрерывной интеграции, что позволяет, например, автоматизировать процессы конвейеризации приложений.',
 'Один из первых алгоритмов разреженной аппроксимации — MP (Matching Pursuit) был разработан Mallat\\xa0S. и Zhang\\xa0Z. в 1993 году [4].',
 'Таким образом, в программном коде появилась возможность использовать следующие конструкции: структура точка {цел а; цел б;}; этого типа: структура точка {цел а; цел б;} т1, т2; структура {цел а; цел б;} т1, т1; структура точка т1;',
 'Лампорта Алгоритм Лампорта отслеживает выполнение отношения предшествования (happensbefore [11]) для инструкций исследуемой программы.',
 'Человек, планируя свой поход, сможет тщательно исследовать местность, оценить время прохождения данного пути, исходя из качества дороги, выбрать места остановок, средство передвижения.',
 'Как уже было сказанно, некоторые задачи обладают большой вычислительной трудоёмкостью, поэтому решения, написанные на P

In [294]:
[sentence for sentence in sentences if len(sentence) > 500]

['Такие функции вызываются посредством транзакций и могут принимать как входные параметры, так и какоето количество токенов1, а также имеют доступ к состоянию блокчейна или смартконтракта и могут вызывать функции в других смартконтрактах, не создавая транзакций, т.е. атомарно.2 Реализация смартконтрактов для расширения возможностей транзакций в сети блокчейн была предложена вместе с Ethereum [2] — платформой для разработки распределенных децентрализованных приложений, которые используют блокчейн для хранения своего состояния.',
 'В качестве целевой платформы, для которой реализуется плагин, была выбрана IntelliJ Platform13, так как она обладает следующими свойствами.  является основой для целого набора IDE, что позволяет использовать плагин в любой из них;  предоставляет не только SDK, но и широкий набор инструментов в виде плагинов, позволяющих облегчить и ускорить разработку плагина;  имеет открытый исходный код платформы и большое количество референсных реализаций других плагинов с 

### habr

In [16]:
habr = pd.read_csv("data/habr/clean_habr.csv",
                 lineterminator='\n')

In [17]:
habr = habr[-5000:]

In [18]:
habr['text'] = habr.text.apply(lambda x: x.replace("\r", "\n "))
habr['text'] = habr.text.apply(lambda x: x.replace("\n", " "))
habr['text'] = habr.text.apply(lambda x: x.replace("\xa0", " "))
habr_texts = habr.text.progress_apply(lambda x: list(sentenize(x)))

100%|██████████| 5000/5000 [00:16<00:00, 297.81it/s]


In [19]:
sentences_habr = list(habr_texts.explode().apply(lambda x: x.text))

In [20]:
sentences_habr

['Предоставленные ниже материалы затрагивают лишь основы, то самое начало, с которого можно стартовать человеку, решившему создать игру но не знающему с чего начать.',
 'Хорошим вариантом будет изучение соответствующей литературы, статей или же рассмотрения примеров уже готовых игр, а так же просмотр исходников разных программ и решений.Данные материалы не претендуют на то, чтобы показать «как надо делать», они лишь помогут наглядно понять, что и как работает, а вот как правильно это сделать — решит только сам разработчик.',
 'Однако, кого-то эти материалы могут замотивировать на дальнейшую работу, так как они достаточно наглядны и объясняют простоту создания игр на JavaScript.1.',
 'Рассмотрение понятия requestAnimationFrame.Для современной игры на JavaScript можно использовать событийную модель поведения, когда игра будет реагировать на события игрока, проявляя свою интерактивность, а можно воспользоваться игровым циклом, который будет отслеживать состояния игровых объектов, следить 

In [21]:
sentences_habr = make_text(sentences_habr)

In [22]:
[sen for sen in sentences_habr if "\xa0" in sen]

[]

In [23]:
len(sentences_habr)

260509

In [24]:
sentences_habr

['Предоставленные ниже материалы затрагивают лишь основы, то самое начало, с которого можно стартовать человеку, решившему создать игру но не знающему с чего начать.',
 'Хорошим вариантом будет изучение соответствующей литературы, статей или же рассмотрения примеров уже готовых игр, а так же просмотр исходников разных программ и решений.Данные материалы не претендуют на то, чтобы показать «как надо делать», они лишь помогут наглядно понять, что и как работает, а вот как правильно это сделать — решит только сам разработчик.',
 'Однако, кого-то эти материалы могут замотивировать на дальнейшую работу, так как они достаточно наглядны и объясняют простоту создания игр на JavaScript.1.',
 'Рассмотрение понятия requestAnimationFrame.Для современной игры на JavaScript можно использовать событийную модель поведения, когда игра будет реагировать на события игрока, проявляя свою интерактивность, а можно воспользоваться игровым циклом, который будет отслеживать состояния игровых объектов, следить 

слипаются предложение у точки
\xa
формулы
много на английском

## combine habr and courseworks

In [26]:
len(sentences)

39618

In [27]:
num_to_select = 39500
coursework_sample = sentences[:num_to_select]
habr_sample = random.sample(sentences_habr, num_to_select)

In [28]:
coursework_sample

['В современном мире становится все более актуальным вопрос мониторинга состояния здоровья человека и принятия мер для его поддержания и укрепления.',
 'Эта тема –– основа для многочисленных исследований научных групп и лабораторий как частных компаний [7], так и подразделений университетов, институтов физической культуры и спорта, ученых в области психологии, и других (например, [12], [11]).',
 'Исследования невозможны без экспериментов и данных, которые специалисты собирают преимущественно в лабораторных условиях с привлечением групп испытуемых – спортсменов или обычных людей разных возрастов.',
 'Часть исследования может состоять в оценке одного или нескольких биометрических маркеров человека в заданных условиях, например, частоты сердечных сокращений, способности удерживать равновесие, объема выдыхаемого воздуха, и так далее.',
 'Лабораторное оборудование позволяет делать измерения с высокой точностью, выполнять соответствующие расчеты, и получать метрики, на которых в дальнейшем у

In [32]:
len(habr_sample)

39500

In [33]:
train_test = 0.25
habr_test_size = int(len(habr_sample)*train_test)
habr_train_size = int(len(habr_sample)-habr_test_size)
coursework_test_size = int(len(coursework_sample)*train_test)
coursework_train_size = int(len(coursework_sample)-coursework_test_size)

coursework_train = coursework_sample[:coursework_train_size]
coursework_test = coursework_sample[coursework_train_size:]
habr_train = habr_sample[:habr_train_size]
habr_test = habr_sample[habr_train_size:]

In [35]:
print(len(habr_train))
print(habr_test_size)
print(habr_test_size + habr_train_size)

29625
9875
39500


In [36]:
train_df = pd.DataFrame({"text": coursework_train+habr_train, "target": coursework_train_size*[1] + habr_train_size*[0]})
test_df = pd.DataFrame({"text": coursework_test+habr_test, "target": coursework_test_size*[1] + habr_test_size*[0]})

In [37]:
test_df

,text,target
0,"Граф взаимодействий состоит из вершин, которые...",1
1,Связь в графе может содержать различные типы в...,1
2,При этом данные в графе можно фильтровать по т...,1
3,Компания Nuix предоставляет криминалистические...,1
4,"Например, Nuix Incident Response содержит визу...",1
...,...,...
19745,Не подрывает ли наше затянувшееся молчание еди...,0
19746,ColumnTypeConverter.cs 91 Весьма интересная о...,0
19747,"Но, как любят говорить некоторые мои знакомые,...",0
19748,Спасибо за внимание.,0


In [38]:
train_df.to_csv("bert_train_58k.csv")
test_df.to_csv("bert_test_58k.csv")